In [8]:
from database.database import Session
from sqlalchemy import select, func
from database.models import Job_1, Job_2, Job_3
from sentence_transformers import SentenceTransformer
import asyncio
import numpy as np 
import pandas as pd

c:\NNL\DS300_Final_Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
user_data = pd.read_csv("Dataset/User_data_final.csv")

In [136]:
user_data

,URL User,UserID,User Name,Industry,Desired Job,Workplace Desired,Desired Salary,Gender,Marriage,Age,Target,Skills,Degree,Work Experience
0,https://timviec365.vn/ung-vien/nguyen-quan-uv9...,964496,Nguyễn Quân,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,36,Với kinh nghiệm 10 năm trong lĩnh vực lái xe t...,"- Bằng lái xe hạng C, D, E và có kinh nghiệm 5...",Đơn vị đào tạo: Trường trung cấp nghề,5-10 năm
1,https://timviec365.vn/ung-vien/nguyen-chau-uv9...,986206,Nguyễn Châu,Vận tải - Lái xe,Nhân viên lái xe bằng D,Bình Dương,Thỏa thuận,Nam,Đã có gia đình,44,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trường trung cấp nghề,Trên 10 năm
2,https://timviec365.vn/ung-vien/la-thi-hoang-uv...,769632,LÃ THỊ HOÀNG,Hành chính - Văn phòng,"Nhân viên văn phòng, khu công nghiệp sử dụng t...",Bắc Giang,Thỏa thuận,Nữ,Độc thân,25,- Tìm kiếm một vị trí nhân viên hành chính và ...,- Am hiểu mọi quy trình về hành chính nhân sự ...,Mô tả công việc: CÔNG TY CỔ PHẦN DƯỢC PHẨM AN ...,1-3 năm
3,https://timviec365.vn/ung-vien/nguyen-van-bao-...,981314,Nguyễn Văn Bảo,Vận tải - Lái xe,Nhân viên lái xe hạng C,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,33,Trở thành kỹ sư tốt trong công tác sửa chữa xe...,"Có khả năng chu toàn công việc, siêng năng, tỉ...",Đơn vị đào tạo: Trung tâm dạy nghề,5-10 năm
4,https://timviec365.vn/ung-vien/nguyen-thi-tu-u...,962892,Nguyễn Thị Tứ,Quản lý đơn hàng,Nhân viên thu mua,Hồ Chí Minh,Thỏa thuận,Nữ,Độc thân,24,"- Có một công việc có nhiều cơ hội thăng tiến,...","- Trung thực, cẩn thận, có óc sáng tạo - Kỹ...",Đơn vị đào tạo: Cao Đẳng Kinh Tế Kỹ Thuật Vina...,3-5 năm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,https://timviec365.vn/ung-vien/bui-nghia-uv997...,997433,Bùi Nghĩa,Vận tải - Lái xe,Nhân viên lái xe hạng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,35,"Trở thành nhân viên lái xe cho công ty, tôi mu...","- Tiếp thu tốt các kiến thức lái xe, - Có trí ...",Đơn vị đào tạo: Trường trung cấp nghề,1-3 năm
3979,https://timviec365.vn/ung-vien/tran-thanh-son-...,964381,Trần Thanh Sơn,Vận tải - Lái xe,Nhân viên Lái xe hạng C,Khánh Hòa,Thỏa thuận,Nam,Đã có gia đình,29,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trung học,5-10 năm
3980,https://timviec365.vn/ung-vien/huynh-qui-uv969...,969486,Huỳnh Quí,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hồ Chí Minh,Thỏa thuận,Nam,Độc thân,20,Áp dụng những kiến thức về giao thông vận tải ...,"Khả năng chịu áp lực cao trong công việc, có t...",Đơn vị đào tạo: Trường trung cấp nghề,Dưới 1 năm
3981,https://timviec365.vn/ung-vien/pham-van-truong...,978744,Phạm Văn Trường,Vận tải - Lái xe,Nhân viên lái xe - Bằng Lái Xe Hạng D,Quảng Ninh,Thỏa thuận,Nam,Đã có gia đình,34,"Đáp ứng tốt các công việc của công ty, giữ gìn...","Giao tiếp tốt, thân thiện, cởi mở với mọi ngườ...",Đơn vị đào tạo: Tốt nghiệp THPT,Trên 10 năm


# Helper Function|

## Khuyến nghị việc làm dựa trên kĩ năng:

In [133]:
async def get_recommendations_based_skill(model, model_name: str, skill: str): 
    # Tính toán embedding cho kỹ năng
    embedding_input = model.encode(skill).tolist()  # Convert tensor/array to list for database compatibility
    
    # Chọn bảng phù hợp dựa trên model_name
    if model_name == "sbert":
        target_table = Job_1
    elif model_name == "phobert":
        target_table = Job_2
    elif model_name == "xlm_roberta":
        target_table = Job_3
    else:
        raise ValueError("Invalid model_name. Supported values are 'sbert', 'phobert' and 'xlm_roberta'.")

    async with Session() as session:
        query = (
            select(
                target_table, 
                target_table.job_skill_vector.cosine_distance(embedding_input).label("distance")
            )
            .where(
                target_table.job_skill != None,
                func.trim(func.regexp_replace(target_table.job_skill, r"[\n\r\u2028]+", "", "g")) != ""
            )
            .order_by("distance")  # Sắp xếp theo khoảng cách cosine
            .limit(20)
        )

        result = await session.execute(query)
        job_results = result.fetchall()

        # Tách job và distance từ kết quả
        jobs = [row[0] for row in job_results]  # `Job_1` or `Job_2` objects
        distances = [row[1] for row in job_results]  # `distance` values
        
        return jobs, distances


## Khuyến nghị việc làm dự trên địa điểm làm việc

In [159]:
async def get_recommendations_based_location(model, model_name: str, location: str):
    # Tính toán embedding cho kỹ năng
    embedding_input = model.encode(location).tolist()  # Convert tensor/array to list for database compatibility

    if model_name == "sbert":
        target_table = Job_1
    elif model_name == "phobert":
        target_table = Job_2
    elif model_name == "xlm_roberta":
        target_table = Job_3
    else:
        raise ValueError("Invalid model_name. Supported values are 'sbert', 'phobert' and 'xlm_roberta'.")
    

    async with Session() as session:
        query = (
            select(target_table,  target_table.job_address_vector.cosine_distance(embedding_input).label("distance"))
            .where(
                target_table.job_address != None,
                func.trim(func.regexp_replace(target_table.job_address, r"[\n\r\u2028]+", "", "g")) != ""
            )
            .order_by("distance")  # Sắp xếp theo khoảng cách cosine
            .limit(20)
        )

        result = await session.execute(query)
        job_results = result.fetchall()

        # Tách job và distance từ kết quả
        jobs = [row[0] for row in job_results]  
        distances = [row[1] for row in job_results]  
        
        return jobs, distances


## Khuyến nghị việc làm dựa trên công việc mơ ước

In [157]:
async def get_recommendations_based_job_title(model, model_name: str, job_title: str):
    # Tính toán embedding cho kỹ năng
    embedding_input = model.encode(job_title).tolist()  # Convert tensor/array to list for database compatibility

    if model_name == "sbert":
        target_table = Job_1
    elif model_name == "phobert":
        target_table = Job_2
    elif model_name == "xlm_roberta":
        target_table = Job_3
    else:
        raise ValueError("Invalid model_name. Supported values are 'sbert', 'phobert' and 'xlm_roberta'.")
    
    async with Session() as session:
        query = (
            select(target_table,  target_table.job_title_vector.cosine_distance(embedding_input).label("distance"))
            .where(
                target_table.job_title != None,
                func.trim(func.regexp_replace(target_table.job_title, r"[\n\r\u2028]+", "", "g")) != ""
            )
            .order_by("distance")  # Sắp xếp theo khoảng cách cosine
            .limit(20)
        )

        result = await session.execute(query)
        job_results = result.fetchall()

        # Tách job và distance từ kết quả
        jobs = [row[0] for row in job_results]  
        distances = [row[1] for row in job_results]  
        
        return jobs, distances


## Khuyến nghị việc làm dựa trên ngành nghề

In [158]:
async def get_recommendations_based_industry(model, model_name: str, industry: str):
    # Tính toán embedding cho kỹ năng
    embedding_input = model.encode(industry).tolist()  # Convert tensor/array to list for database compatibility

    if model_name == "sbert":
        target_table = Job_1
    elif model_name == "phobert":
        target_table = Job_2
    elif model_name == "xlm_roberta":
        target_table = Job_3
    else:
        raise ValueError("Invalid model_name. Supported values are 'sbert', 'phobert' and 'xlm_roberta'.")
    
    async with Session() as session:
        query = (
            select(target_table,  target_table.industry_vector.cosine_distance(embedding_input).label("distance"))
            .where(
                target_table.industry != None,
                func.trim(func.regexp_replace(target_table.industry, r"[\n\r\u2028]+", "", "g")) != ""
            )
            .order_by("distance")  # Sắp xếp theo khoảng cách cosine
            .limit(20)
        )

        result = await session.execute(query)
        job_results = result.fetchall()

        # Tách job và distance từ kết quả
        jobs = [row[0] for row in job_results]  
        distances = [row[1] for row in job_results]  
        
        return jobs, distances


## Khuyến nghị dựa trên cả 4 yếu tố:

In [160]:
async def get_recommendations_based_combined(model, model_name: str, industry: str, job_title: str, job_skill: str, job_address: str):
    # Tính toán embedding cho kỹ năng
    industry_input = model.encode(industry).tolist()  # Convert tensor/array to list for database compatibility
    job_title_input = model.encode(job_title).tolist()
    job_skill_input = model.encode(job_skill).tolist()
    job_address_input = model.encode(job_address).tolist()

    if model_name == "sbert":
        target_table = Job_1
    elif model_name == "phobert":
        target_table = Job_2
    elif model_name == "xlm_roberta":
        target_table = Job_3
    else:
        raise ValueError("Invalid model_name. Supported values are 'sbert', 'phobert' and 'xlm_roberta'.")

    async with Session() as session:
        query = (
            select(target_table,  
                   target_table.job_title_vector.cosine_distance(job_title_input).label("job_title_distance"),
                   target_table.industry_vector.cosine_distance(industry_input).label("industry_distance"),
                   target_table.job_skill_vector.cosine_distance(job_skill_input).label("job_skill_distance"),
                   target_table.job_address_vector.cosine_distance(job_address_input).label("job_address_distance")
            )
            .where(
                target_table.industry != None,
                target_table.job_title != None,
                target_table.job_skill != None,
                target_table.job_address != None,
                func.trim(func.regexp_replace(target_table.industry, r"[\n\r\u2028]+", "", "g")) != "",
                func.trim(func.regexp_replace(target_table.job_title, r"[\n\r\u2028]+", "", "g")) != "",
                func.trim(func.regexp_replace(target_table.job_skill, r"[\n\r\u2028]+", "", "g")) != "",
                func.trim(func.regexp_replace(target_table.job_address, r"[\n\r\u2028]+", "", "g")) != "",
            )
            .order_by("job_title_distance", "industry_distance",  "job_skill_distance", "job_address_distance")  # Sắp xếp theo khoảng cách cosine
            .limit(20)
        )

        result = await session.execute(query)
        job_results = result.fetchall()

        # Tách job và distance từ kết quả
        jobs = [row[0] for row in job_results]  # `Job_1` objects
        industry_distances = [row[1] for row in job_results]  # `distance` values
        job_title_distances = [row[2] for row in job_results]
        job_skill_distances = [row[3] for row in job_results]
        job_address_distances = [row[4] for row in job_results]
        
        return jobs, industry_distances, job_title_distances, job_skill_distances, job_address_distances


In [185]:
async def Get_multi_recommendations(df, model, model_name: str, method: str):
    """
    Mở rộng thông tin gợi ý công việc với khoảng cách cho mỗi người trong DataFrame.
    Dựa trên phương thức ('industry' hoặc 'skill'), sẽ gọi hàm thích hợp.

    Args:
        df (pd.DataFrame): DataFrame chứa thông tin người dùng.
        model: Mô hình để tính toán embedding ngành nghề hoặc kỹ năng.
        model_name (str): Tên của mô hình (e.g., "sbert", "phobert").
        method (str): Phương thức gọi hàm (có thể là 'industry' hoặc 'skill').

    Returns:
        pd.DataFrame: DataFrame mở rộng với các công việc gợi ý và khoảng cách cho từng người.
    """
    # Giới hạn dataframe chỉ lấy 10 người đầu tiên
    df = df.sample(n=10, random_state=42)

    # Danh sách kết quả mở rộng
    expanded_results = []

    # Duyệt qua từng người dùng trong dataframe
    for _, row in df.iterrows():
        # Lấy thông tin ngành nghề
        
        try:
            # Gọi hàm gợi ý công việc dựa trên phương thức (industry hoặc skill)
            if method == "industry":
                jobs, distances = await get_recommendations_based_industry(model, model_name, row["Industry"])
            elif method == "skill":
                jobs, distances = await get_recommendations_based_skill(model, model_name, row["Skills"])
            elif method == "location":
                jobs, distances = await get_recommendations_based_location(model, model_name, row["Workplace Desired"])
            elif method == "job title":
                jobs, distances = await get_recommendations_based_job_title(model, model_name, row["Desired Job"])
            elif method == "combined":
                jobs, distances, job_title_distances, job_skill_distances, job_address_distances = await get_recommendations_based_combined(model, model_name, row["Industry"], row["Desired Job"], row["Skills"], row["Workplace Desired"])
            else:
                raise ValueError("Method must be either 'industry' or 'skill'.")

            # Tạo một dòng riêng biệt cho mỗi công việc gợi ý
            for job, distance in zip(jobs, distances):
                expanded_results.append({
                    "User ID": row.get("UserID", None),
                    "User Name": row.get("User Name", None),
                    "Industry": row.get("Industry", None),
                    "Age": row.get("Age", None),
                    "Job Title Recommended": job.job_title,
                    "Company Name": job.company_name,
                    "Industry Recommended": job.industry,
                    "Skills Recommended": job.job_skill,
                    "Address Recommended": job.job_address,
                    "Job Requierements": job.job_requirements,
                    "Job Description": job.job_description,
                    "Job Salary": job.salary,
                    "Job Years of Experience": job.years_of_experience,
                    "Distance": distance
                })
        except Exception as e:
            print(e)

    # Chuyển kết quả mở rộng thành dataframe
    expanded_df = pd.DataFrame(expanded_results)
    return expanded_df


# Models

## sBERT

In [208]:
model = SentenceTransformer("keepitreal/vietnamese-sbert")

In [209]:
skill_recommendation_results = await Get_multi_recommendations(user_data, model, "sbert", "skill")
skill_recommendation_results

2024-12-16 01:01:26,033 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:01:26,037 INFO sqlalchemy.engine.Engine SELECT jobs_1.id, jobs_1.url, jobs_1.job_title, jobs_1.company_name, jobs_1.industry, jobs_1.company_address, jobs_1.job_description, jobs_1.job_requirements, jobs_1.benefits, jobs_1.salary, jobs_1.expire_date, jobs_1.employee_rank, jobs_1.job_skill, jobs_1.job_field, jobs_1.years_of_experience, jobs_1.job_address, jobs_1.job_title_vector, jobs_1.industry_vector, jobs_1.job_skill_vector, jobs_1.job_address_vector, jobs_1.job_skill_vector <=> $1 AS distance 
FROM jobs_1 
WHERE jobs_1.job_skill IS NOT NULL AND trim(regexp_replace(jobs_1.job_skill, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:01:26,040 INFO sqlalchemy.engine.Engine [cached since 5974s ago] ('[0.03798454627394676,0.3454383909702301,0.017232272773981094,0.015160647220909595,0.2480212152004242,0.8499230146408081,0.041743144392967224,-0.11242

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Phiên Dịch Tiếng Nhật,Công Ty TNHH YKK Việt Nam,Dệt may/May mặc/Giày dép,"Kỹ năng giao tiêp, Kỹ năng Tin học văn phòng, ...","KCN Đồng Văn 3, Phường Hoàng Đông, Thị xã Duy ...",1. Bằng cấp: Tốt nghiệp Cao đẳng trở lên (chuy...,"MÔ TẢ CÔNG VIỆC. Làm các thủ tục, giấy tờ cho ...",Thương lượng,1-3 năm,0.312839
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Nhóm Sự Kiện,Hệ thống Giáo Dục Edufit,Giáo dục/Đào Tạo,"Kỹ năng giao tiếp, Kỹ năng giao tiếp, Kỹ năng ...","lô PT-01, KĐT Vinhomes Ocean Park, Đa Tốn, Gia...",Tốt nghiệp đại học trở lên. Khả năng sử dụng t...,Tổ chức/Thực hiện/Giám sát/Quản lý công việc S...,"Trên 40,000,000",1-3 năm,0.355655
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Đào Tạo,Công Ty Vàng Bạc Đá Quý Bảo Tín Minh Châu,Bán lẻ/Bán sỉ,"Kỹ năng giao tiếp, Kỹ năng lập kế hoạch, Soạn ...",139 Cầu Giấy,"Giới tính: Nam và Nữ, độ tuổi từ 35 – 45 tuổi;...",I. MÔ TẢ CÔNG VIỆC:. Phụ trách tổ chức vận hàn...,"20,000,000 - 40,000,000",3-5 năm,0.365367
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Nhóm Chất Lượng (QA),Alphanam Group,Kỹ thuật xây dựng/Cơ sở hạ tầng,"Kỹ năng giao tiếp, Kỹ năng giao tiếp, Kỹ năng ...","108 Nguyễn Trãi, Thượng Đình, Thanh Xuân, Hà N...",Tốt nghiệp Đại học trở lên các chuyên ngành: Q...,* Xây dựng hệ thống văn bản chất lượng (Quy tr...,Thương lượng,5-10 năm,0.414016
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Bộ phận Mua Hàng,Công Ty Cổ Phần Ánh Dương Building (ADB),Vật liệu xây dựng,"Kỹ năng giao tiếp, Kỹ năng giao tiếp, Làm việc...","Cao ốc An Khang Building, Số 28, Đường số 19, ...",2. Yêu cầu tuyển dụng:. 2.1. Trình độ:. Nam/nữ...,1. Trách nhiệm công việc:. Xây dựng kế hoạch m...,"Trên 40,000,000",3-5 năm,0.420976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Kế Toán Tổng Hợp,Cty TNHH Quốc Tế Anh Duy,Nhập khẩu/Xuất khẩu,"Lập Báo Cáo Tài Chính, Kế Toán Tổng Hợp, Phần ...","170 Ung Văn Khiêm, Phường 25, Bình Thạnh, Thàn...",Yêu cầu: Nữ; Tốt nghiệp Đại Học/Cao đẳng. Thàn...,Kiểm tra đối chiếu số liệu giữa các đơn vị nội...,Thương lượng,1-3 năm,0.458464
196,973260,Lê Thanh,Vận tải - Lái xe,24,Quản Lý Điều Hành Doanh Nghiệp (Operations Man...,CÔNG TY TNHH SMART OUTSOURCING,Kế toán/Kiểm toán,"Lập Kế Hoạch Tài Chính, MBA, Quản Lý Điều Hành...","Tầng 9, Tòa văn phòng C1 Thành Công, phường Th...",III. YÊU CẦU. Tốt nghiệp trình độ Đại họ...,I . KẾT QUẢ CÔNG VIỆC CẦN ĐẠT. Giám sát và quả...,"15,000,000 - 20,000,000",1-3 năm,0.459906
197,973260,Lê Thanh,Vận tải - Lái xe,24,Kế Toán Nội Bộ,Công Ty Cổ Phần Vận Tải Biển Container Thái Bì...,Hậu cần/Giao nhận,"Kế Toán, Kế Toán Nội Bộ, Báo Cáo Tài Chính, Ph...","Tầng 15 tòa nhà Hòa Bình, 106 Hoàng Quốc Việt,...",YÊU CẦU CÔNG VIỆC:. + Tốt nghiệp ĐH chuyên ngà...,MÔ TẢ CÔNG VIỆC:. Nhập dữ liệu định khoản lên ...,"10,000,000 - 15,000,000",3-5 năm,0.460947
198,973260,Lê Thanh,Vận tải - Lái xe,24,Xưởng Trưởng Bảo Dưỡng Sửa Chữa Ô Tô - Hồ Chí ...,Công Ty TNHH Dịch Vụ Vận Tải Sinh Thái Vinbus,Ô tô,"Quản Trị Nhân Sự, Sửa Chữa Bảo Dưỡng, Điều Hàn...","KĐT Vinhomes Grand Park, Số 10 đường T10 phân ...","Bằng cấp chuyên môn. Tốt nghiệp Đại học, Cao đ...","Chịu trách nhiệm toàn bộ về công tác tổ chức, ...","Trên 40,000,000",3-5 năm,0.463100


In [210]:
address_recommendation_results = await Get_multi_recommendations(user_data, model, "sbert", "location")
address_recommendation_results

2024-12-16 01:01:44,450 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:01:44,453 INFO sqlalchemy.engine.Engine SELECT jobs_1.id, jobs_1.url, jobs_1.job_title, jobs_1.company_name, jobs_1.industry, jobs_1.company_address, jobs_1.job_description, jobs_1.job_requirements, jobs_1.benefits, jobs_1.salary, jobs_1.expire_date, jobs_1.employee_rank, jobs_1.job_skill, jobs_1.job_field, jobs_1.years_of_experience, jobs_1.job_address, jobs_1.job_title_vector, jobs_1.industry_vector, jobs_1.job_skill_vector, jobs_1.job_address_vector, jobs_1.job_address_vector <=> $1 AS distance 
FROM jobs_1 
WHERE jobs_1.job_address IS NOT NULL AND trim(regexp_replace(jobs_1.job_address, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:01:44,455 INFO sqlalchemy.engine.Engine [cached since 4353s ago] ('[-0.2742690145969391,0.4564611315727234,-0.6578325629234314,-0.35101208090782166,-0.26071643829345703,-0.12473781406879425,0.3879827558994293,-

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Phó Quản Lý Chi Nhánh Thẩm Mỹ,Công Ty TNHH Dược Mỹ Phẩm H-Derma,Sản xuất/Vận hành sản xuất,"Mỹ Phẩm, Giám Sát Bán Hàng, Quản Lý Điều Hành",Khánh Hòa,Tối thiểu trên 01 năm kinh nghiệm làm vị trí t...,"Xây dựng, tổ chức, vận hành giám sát công việc...","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.000000
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Phụ Trách Kinh Doanh,Công Ty TNHH Thép Tân Thắng Lợi,Kinh doanh,"Tư Vấn Bán Hàng, Phát Triển Thị Trường",Khánh Hòa,Tốt nghiệp cao đẳng trở lên chuyên ngành Thươn...,"Xây dựng, tìm các đối tác khách hàngBáo giá kh...","15,000,000 - 20,000,000",Không yêu cầu kinh nghiệm,0.000000
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Vận Hành Dịch Vụ,Công Ty Cổ Phần Phần Dịch Vụ Bạn Uống Tôi Lái,Kinh doanh,"Quản Trị Hệ Thống, Hành chính Văn phòng","|,Khánh Hòa","Yêu cầu ứng viên:Tốt nghiệp trung cấp, cao đẳn...",Chủ động liên hệ với khách hàng để giải quyết ...,"5,000,000 - 10,000,000",Không yêu cầu kinh nghiệm,0.144465
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Bán Hàng Vùng Kênh Truyền Thống - Area ...,Công Ty Cổ Phần Sữa Việt Nam (Vinamilk),Hàng tiêu dùng,"Direct Sales, General Trade","Khánh Hòa, Vietnam",HỌC VẤN & CHUYÊN NGÀNH:. Tốt nghiệp đại học. Ư...,KHU VỰC PHỤ TRÁCH: Các tỉnh từ Khánh Hòa vào đ...,Thương lượng,5-10 năm,0.201885
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,New Product Introduction Engineer - NPI Engineer,Onsemi Viet Nam,Sản xuất,"Electronics, New Product Qualification, Lean S...",Biên Hòa,"Qualifications. BS/MS in Electrical, Mechanica...",Job Description. Introduces and maintains asse...,Thương lượng,Không yêu cầu kinh nghiệm,0.260227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Kỹ Thuật IT Sửa PC Laptop Máy In,Công Ty TNHH Phát Triển Công Nghệ Trường Thịnh...,"Công nghệ thông tin, Quảng cáo/Marketing/PR","IT Helpdesk, IT Support",Hồ Chí Minh,Có Ít nhất > 1 năm KN làm chuyên viên kỹ thuật...,"Mua bán sửa chữa máy tính bàn pc, laptop, máy ...","5,000,000 - 10,000,000",1-3 năm,0.000000
196,973260,Lê Thanh,Vận tải - Lái xe,24,- Nhân Viên Phụ Bếp Part Time,Công ty Cổ phần Đầu tư Thương mại Quốc tế Mặt ...,Thực phẩm/DV ăn uống,"Phụ Bếp, Chế Biến Thực Phẩm, Tư Vấn Bán Hàng",Hồ Chí Minh,Yêu Cầu Công ViệcLàm việc tại: Nhà hàng Hotpot...,Thực hiện sơ chế các nguyên vật liệu dùng đ...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.000000
197,973260,Lê Thanh,Vận tải - Lái xe,24,Quản Lý Kinh Doanh Thu Nhập Từ 20TR,Công Ty TNHH Bảo Hiểm Nhân Thọ Cathay - Việt Nam,bảo hiểm,"Chăm Sóc Khách Hàng, Tư Vấn Bán Hàng, Tư Vấn B...",Hồ Chí Minh,Tốt nghiệp trung cấp trở lên trên 26 tuổi. Đã ...,"Quản lý, đào tạo nhân viên, hỗ trợ tuyển dụng....","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.000000
198,973260,Lê Thanh,Vận tải - Lái xe,24,Store Manager - Cửa Hàng Trưởng,Công ty TNHH Manpower Việt Nam - Hồ Chí Minh,Nhân Sự,"Quản Lý Điều Hành, Giám Sát Bán Hàng, Bán Lẻ",Hồ Chí Minh,At least 1 year of experience in fashion retai...,Stock & Sales Management:. Ensure effective re...,"20,000,000 - 40,000,000",1-3 năm,0.000000


In [211]:
industry_recommendation_results = await Get_multi_recommendations(user_data, model, "sbert", "industry")
industry_recommendation_results

2024-12-16 01:01:49,068 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:01:49,070 INFO sqlalchemy.engine.Engine SELECT jobs_1.id, jobs_1.url, jobs_1.job_title, jobs_1.company_name, jobs_1.industry, jobs_1.company_address, jobs_1.job_description, jobs_1.job_requirements, jobs_1.benefits, jobs_1.salary, jobs_1.expire_date, jobs_1.employee_rank, jobs_1.job_skill, jobs_1.job_field, jobs_1.years_of_experience, jobs_1.job_address, jobs_1.job_title_vector, jobs_1.industry_vector, jobs_1.job_skill_vector, jobs_1.job_address_vector, jobs_1.industry_vector <=> $1 AS distance 
FROM jobs_1 
WHERE jobs_1.industry IS NOT NULL AND trim(regexp_replace(jobs_1.industry, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:01:49,072 INFO sqlalchemy.engine.Engine [cached since 631.6s ago] ('[0.44310906529426575,0.04457218572497368,-0.14286266267299652,-0.24837009608745575,-0.1243777871131897,-0.26507705450057983,-0.1482895314693451,-0.58 .

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Digital Marketing Hỗ Trợ Vận Hành Sà...,Công Ty Cổ Phần Thương Mại Thiết Bị Văn Phòng ...,Trang thiết bị văn phòng,"Social Marketing, Digital Marketing, Truyền Th...","43 Linh Đông, Khu Phố 7, P. Linh Đông, Tp Thủ Đức","Tốt nghiệp Cao đẳng chuyên ngành Marketing, bá...","Lập kế hoạch, phối hợp tổ chức và triển khai c...",Thỏa thuận,1-3 năm,0.430879
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Tư Vấn Văn Phòng (Thu Nhập Đến 20 Tr...,Công Ty TNHH Một Thành Viên Đức Lan,Trang thiết bị văn phòng,"Tư Vấn Doanh Nghiệp (B2B), Tư Vấn Bán Hàng, Bá...","số 149 Võ Văn Tần, Võ Thị Sáu, Quận 3, Hồ Chí ...","Nữ, Tốt nghiệp trung cấp trở lênƯu tiên có kin...","Công ty chuyên phân phối và cung cấp Máy, Mực,...","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.430879
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Phó Bộ Phận QC,Công Ty Cổ Phần Văn Phòng Phẩm Hải Phòng,Trang thiết bị văn phòng,"Quản Lý Chất Lượng (QA/QC), Quản Lý Sản Xuất",Tổ 1 – KDC Phương Lung – Đường Mạc Đăng Doanh ...,Nam/Nữ từ 25 40 tuổi;. Tốt nghiệp Đại học các ...,"Quản lý, phân công công việc cho nhân viên;. K...",Thỏa thuận,1-3 năm,0.430879
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Kinh Doanh (Thu Nhập Đến 20 Triệu),Công Ty TNHH Một Thành Viên Đức Lan,Trang thiết bị văn phòng,"Tư Vấn Doanh Nghiệp (B2B), Tư Vấn Bán Hàng, Bá...","số 149 Võ Văn Tần, Võ Thị Sáu, Quận 3, Hồ Chí ...","Nữ, Tốt nghiệp trung cấp trở lênƯu tiên có kin...","Công ty chuyên phân phối và cung cấp Máy, Mực,...","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.430879
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Lập Trình Mobile (Flutter Dev),Công Ty TNHH Công Nghệ Katec,"Kinh doanh, Trang thiết bị văn phòng","Android, iOS, Lập trình Mobile, UI/UX","189 PHAN HUY CHÚ, AN KHÁNH, NINH KIỀU, TP CT,C...",Có ít nhất 1 năm kinh nghiệm làm việc về phát ...,Tham gia phát triển các dự án của công ty.Xây ...,"5,000,000 - 10,000,000",1-3 năm,0.444103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Customs Clearance Manager,Samsung SDS,Vận tải,Electronics,"SEHC, Cổng số 1, Tăng Nhơn Phú B, Quận 9, Thàn...",(1)EDUCATION / QUALIFICATION:. College or univ...,(1) BROAD SCOPE:. Consulting of customs cleara...,"Trên 40,000,000",Trên 10 năm,0.221493
196,973260,Lê Thanh,Vận tải - Lái xe,24,Account Executive,"FedEx Express Vietnam Co., Ltd.",Vận tải,"Communications, Negotiation","15F, 4 Ton That Tung Street, Trung Tu Ward, Do...","To do the job well, you will need the followin...","In the position of an Account Executive, your ...",Thương lượng,1-3 năm,0.221493
197,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Khai Thác Sân Bay Nội Bài Không Yêu ...,Công Ty Cổ Phần Nhà Ga Hàng Hóa ALS,Vận tải,"Quản Lý Kho, Xuất Nhập Khẩu, Giao Nhận Kho Vận...","Tầng 4, ga hàng hóa ALS, Cảng HKQT Nội bài, Ph...",Trình độ học vấn: Tốt nghiệp các trường Đại họ...,* Nhân viên sẽ phụ trách làm một số mảng công ...,"Trên 40,000,000",Không yêu cầu kinh nghiệm,0.221493
198,973260,Lê Thanh,Vận tải - Lái xe,24,Hà Nội - Chuyên Viên Kinh Doanh (Bán Xe Ô Tô V...,GSM,Vận tải,"Kinh doanh ô tô, Sales, Marketing","Tầng B1 tòa R3 Royal City, 72 Nguyễn Trãi, Tha...",Tốt nghiệp từ Trung cấp trở lên. Ít nhất 1 năm...,"Tìm kiếm Khách hàng, triển khai bán hàng, hoàn...","Trên 40,000,000",Không yêu cầu kinh nghiệm,0.221493


In [212]:
job_title_recommendation_results = await Get_multi_recommendations(user_data, model, "sbert", "job title")
job_title_recommendation_results

2024-12-16 01:01:54,561 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:01:54,565 INFO sqlalchemy.engine.Engine SELECT jobs_1.id, jobs_1.url, jobs_1.job_title, jobs_1.company_name, jobs_1.industry, jobs_1.company_address, jobs_1.job_description, jobs_1.job_requirements, jobs_1.benefits, jobs_1.salary, jobs_1.expire_date, jobs_1.employee_rank, jobs_1.job_skill, jobs_1.job_field, jobs_1.years_of_experience, jobs_1.job_address, jobs_1.job_title_vector, jobs_1.industry_vector, jobs_1.job_skill_vector, jobs_1.job_address_vector, jobs_1.job_title_vector <=> $1 AS distance 
FROM jobs_1 
WHERE jobs_1.job_title IS NOT NULL AND trim(regexp_replace(jobs_1.job_title, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:01:54,567 INFO sqlalchemy.engine.Engine [cached since 4528s ago] ('[0.34603989124298096,0.04691627249121666,-0.0481988899409771,-0.07198398560285568,-0.26271557807922363,-0.096248559653759,-0.023860599845647812,-0.15

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính - Nhân Sự,Viện Nghiên Cứu Và Đào Tạo Bất Động Sản,Kinh doanh,"Đào Tạo, Quản Lý Điều Hành, Quản Trị Nhân Sự (...","Đà Nẵng: 136 Tống Phước Phổ, Hải Châu",Tốt nghiệp Đại học chuyên ngành Kinh tế/ Quản ...,Thực hiện công tác phát triển nhân sự.Triển kh...,Thỏa thuận,1-3 năm,0.179899
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính - Nhân Sự,Công Ty TNHH Thương Mại Và Dịch Vụ Tocepo,Kinh doanh,Hành chính Văn phòng,"Số 224 Đống Đa, P. Thị Nại, TP. Qui Nhơn, tỉnh...",Có bằng Cử nhân trở lên thuộc chuyên ngành Quả...,"Quản lý hệ thống các chính sách, quy định, quy...",Thỏa thuận,1-3 năm,0.179899
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính Nhân Sự,Công Ty CP Thực Phẩm Richy Miền Nam,Sản xuất,"Tuyển Dụng, Quản Trị Nhân Sự, Nhân Sự, Hành Ch...","Tầng 8, Tòa Victory Tower, Số 12 Tân Trào, P T...",Tốt nghiệp Đại học trở lên. Ứng viên 03 năm ki...,1. Nhân sự:. a. Xây dựng và quản trị hệ thống:...,"20,000,000 - 40,000,000",3-5 năm,0.188543
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính Nhân Sự,Công Ty TNHH Thời Trang Tracy,Kinh doanh,Hành chính Văn phòng,"141 Bà Triệu, Hai Bà Trưng",●Giới tính: Nữ. ●Độ tuổi: Sinh từ 1985 đến 199...,Công tác hành chính. ● Quản lý và phê duyệt cô...,Thỏa thuận,5-10 năm,0.188543
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính Nhân Sự,Công Ty TNHH Kim Hà Việt,Sản xuất,"Quản Trị Nhân Sự, HR, Tuyển Dụng, Hành Chính N...",Tra Vinh,Tốt nghiệp đại học các chuyên ngành Quản trị N...,"1. Công tác nhân sự:. Tổ chức, quản lý công tá...","Trên 40,000,000",5-10 năm,0.188543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,QA Executive (Ưu Tiên Có Bằng Lái Xe B2),Rentokil Initial (Viet Nam) Limited Company,Dịch vụ môi trường/Chất thải,"Quality Assurance, Quality Control, Pest Contr...","P. Hưng Phú, Quận Cái Răng, TP. Cần Thơ","Graduated from college and above, chemical or ...",1. Inspection & Reporting. To constantly monit...,"Trên 40,000,000",1-3 năm,0.482940
196,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Lái Xe Sân Đỗ,Công ty TNHH Một thành viên Kỹ thuật Máy bay -...,Cơ khí/Kĩ thuật ứng dụng,"Lái Xe, Lái Xe Tải","Sân bay quốc tế Nội Bài, Sóc Sơn, Hà Nội|",Là công dân Việt Nam; từ đủ 18 tuổi đến 45 tuổ...,Lái xe trên sân đỗ máy bay phục vụ công tác bả...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.492187
197,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Bán Hàng Mercedes-Benz Khu Vực Hà Nội,Mercedes-Benz Haxaco Láng Hạ,Ô tô,"Kinh doanh ô tô, Bán Hàng Cao Cấp, Ô Tô, Bán H...","46 Phố Láng Hạ, phường Láng Hạ, Đống Đa, Hà Nộ...",Kinh nghiệm: Có từ 12 năm kinh nghiệm bán hàng...,Chúng tôi tìm kiếm nhân sự vàng cho vị trí Nhâ...,"Trên 40,000,000",Không yêu cầu kinh nghiệm,0.500344
198,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Lái Xe Bằng Lái E,Công Ty Du Lịch Hưng Long,Du lịch,"Lái Xe, Lái Xe Khách","338 Trần Khát Chân, Thanh Nhàn, Hai Bà Trưng, ...",Bằng lái: E ( ưu tiên có kinh nghiệm lái xe kh...,"Phục vụ, chăn sóc khách hàng trong suốt chuyến...","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.501657


In [213]:
combined_recommendation_results = await Get_multi_recommendations(user_data, model, "sbert", "combined")
combined_recommendation_results

2024-12-16 01:01:58,963 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:01:58,972 INFO sqlalchemy.engine.Engine SELECT jobs_1.id, jobs_1.url, jobs_1.job_title, jobs_1.company_name, jobs_1.industry, jobs_1.company_address, jobs_1.job_description, jobs_1.job_requirements, jobs_1.benefits, jobs_1.salary, jobs_1.expire_date, jobs_1.employee_rank, jobs_1.job_skill, jobs_1.job_field, jobs_1.years_of_experience, jobs_1.job_address, jobs_1.job_title_vector, jobs_1.industry_vector, jobs_1.job_skill_vector, jobs_1.job_address_vector, jobs_1.job_title_vector <=> $1 AS job_title_distance, jobs_1.industry_vector <=> $2 AS industry_distance, jobs_1.job_skill_vector <=> $3 AS job_skill_distance, jobs_1.job_address_vector <=> $4 AS job_address_distance 
FROM jobs_1 
WHERE jobs_1.industry IS NOT NULL AND jobs_1.job_title IS NOT NULL AND jobs_1.job_skill IS NOT NULL AND jobs_1.job_address IS NOT NULL AND trim(regexp_replace(jobs_1.industry, $5::VARCHAR, $6::VARCHAR, $7::VARCHAR)) != $8::VAR

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính - Nhân Sự,Viện Nghiên Cứu Và Đào Tạo Bất Động Sản,Kinh doanh,"Đào Tạo, Quản Lý Điều Hành, Quản Trị Nhân Sự (...","Đà Nẵng: 136 Tống Phước Phổ, Hải Châu",Tốt nghiệp Đại học chuyên ngành Kinh tế/ Quản ...,Thực hiện công tác phát triển nhân sự.Triển kh...,Thỏa thuận,1-3 năm,0.179899
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính - Nhân Sự,Công Ty TNHH Thương Mại Và Dịch Vụ Tocepo,Kinh doanh,Hành chính Văn phòng,"Số 224 Đống Đa, P. Thị Nại, TP. Qui Nhơn, tỉnh...",Có bằng Cử nhân trở lên thuộc chuyên ngành Quả...,"Quản lý hệ thống các chính sách, quy định, quy...",Thỏa thuận,1-3 năm,0.179899
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính Nhân Sự,Công Ty TNHH Kim Hà Việt,Sản xuất,"Quản Trị Nhân Sự, HR, Tuyển Dụng, Hành Chính N...",Tra Vinh,Tốt nghiệp đại học các chuyên ngành Quản trị N...,"1. Công tác nhân sự:. Tổ chức, quản lý công tá...","Trên 40,000,000",5-10 năm,0.188543
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính Nhân Sự,Công Ty CP Thực Phẩm Richy Miền Nam,Sản xuất,"Tuyển Dụng, Quản Trị Nhân Sự, Nhân Sự, Hành Ch...","Tầng 8, Tòa Victory Tower, Số 12 Tân Trào, P T...",Tốt nghiệp Đại học trở lên. Ứng viên 03 năm ki...,1. Nhân sự:. a. Xây dựng và quản trị hệ thống:...,"20,000,000 - 40,000,000",3-5 năm,0.188543
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Trưởng Phòng Hành Chính Nhân Sự,Công Ty Cổ Phần Hóa Dầu Vĩnh Phúc,Kinh doanh,"Quản Trị Nhân Sự (HR), Bảo Hiểm Và Tiền Lương ...","10A/13, QL1A, Tân Hưng, Tân Hạnh, Long Hồ, Vĩn...",· Có bằng cử nhân trở lên chuyên ngành Kinh tế...,· Xây dựng kế hoạch tuyển dụng. · Phối hợp các...,"10,000,000 - 15,000,000",1-3 năm,0.188543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,QA Executive (Ưu Tiên Có Bằng Lái Xe B2),Rentokil Initial (Viet Nam) Limited Company,Dịch vụ môi trường/Chất thải,"Quality Assurance, Quality Control, Pest Contr...","P. Hưng Phú, Quận Cái Răng, TP. Cần Thơ","Graduated from college and above, chemical or ...",1. Inspection & Reporting. To constantly monit...,"Trên 40,000,000",1-3 năm,0.482940
196,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Lái Xe Sân Đỗ,Công ty TNHH Một thành viên Kỹ thuật Máy bay -...,Cơ khí/Kĩ thuật ứng dụng,"Lái Xe, Lái Xe Tải","Sân bay quốc tế Nội Bài, Sóc Sơn, Hà Nội|",Là công dân Việt Nam; từ đủ 18 tuổi đến 45 tuổ...,Lái xe trên sân đỗ máy bay phục vụ công tác bả...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.492187
197,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Bán Hàng Mercedes-Benz Khu Vực Hà Nội,Mercedes-Benz Haxaco Láng Hạ,Ô tô,"Kinh doanh ô tô, Bán Hàng Cao Cấp, Ô Tô, Bán H...","46 Phố Láng Hạ, phường Láng Hạ, Đống Đa, Hà Nộ...",Kinh nghiệm: Có từ 12 năm kinh nghiệm bán hàng...,Chúng tôi tìm kiếm nhân sự vàng cho vị trí Nhâ...,"Trên 40,000,000",Không yêu cầu kinh nghiệm,0.500344
198,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Lái Xe Bằng Lái E,Công Ty Du Lịch Hưng Long,Du lịch,"Lái Xe, Lái Xe Khách","338 Trần Khát Chân, Thanh Nhàn, Hai Bà Trưng, ...",Bằng lái: E ( ưu tiên có kinh nghiệm lái xe kh...,"Phục vụ, chăn sóc khách hàng trong suốt chuyến...","20,000,000 - 40,000,000",Không yêu cầu kinh nghiệm,0.501657


In [229]:
industry_recommendation_results.to_csv("results/sbert/industry_recommendation_results.csv", index=False)
skill_recommendation_results.to_csv("results/sbert/skill_recommendation_results.csv", index=False)
address_recommendation_results.to_csv("results/sbert/address_recommendation_results.csv", index=False)
job_title_recommendation_results.to_csv("results/sbert/job_title_recommendation_results.csv", index=False)
combined_recommendation_results.to_csv("results/sbert/combined_recommendation_results.csv", index=False)

## PhoBERT

In [215]:
model = SentenceTransformer('vinai/phobert-base')

No sentence-transformers model found with name vinai/phobert-base. Creating a new one with mean pooling.


In [231]:
skill_recommendation_results = await Get_multi_recommendations(user_data, model, "phobert", "skill")
skill_recommendation_results

2024-12-16 01:09:39,816 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:09:39,827 INFO sqlalchemy.engine.Engine SELECT jobs_2.id, jobs_2.url, jobs_2.job_title, jobs_2.company_name, jobs_2.industry, jobs_2.company_address, jobs_2.job_description, jobs_2.job_requirements, jobs_2.benefits, jobs_2.salary, jobs_2.expire_date, jobs_2.employee_rank, jobs_2.job_skill, jobs_2.job_field, jobs_2.years_of_experience, jobs_2.job_address, jobs_2.job_title_vector, jobs_2.industry_vector, jobs_2.job_skill_vector, jobs_2.job_address_vector, jobs_2.job_skill_vector <=> $1 AS distance 
FROM jobs_2 
WHERE jobs_2.job_skill IS NOT NULL AND trim(regexp_replace(jobs_2.job_skill, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:09:39,830 INFO sqlalchemy.engine.Engine [generated in 0.00481s] ('[-0.018804701045155525,0.015157408080995083,0.025009622797369957,0.02211725153028965,0.14088299870491028,0.023517217487096786,-0.019457638263702393,0 

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân viên đóng gói theo giờ,Công Ty Cổ Phần Gbc Global,Kinh doanh,"Phụ Kho, Công Nhân","Biệt thự Khu đô thị Văn Phú, Hà Đông, Hà Nội","Cẩn thận, tỉ mỉ, trung thực và có trách nhiệm ...",Đóng gói giỏ hàng Tết. Thực hiện công việc đón...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.877040
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,R&D Executive (Thiết Bị Xe Ô Tô),Vietmap - Công ty TNHH Ứng Dụng Bản Đồ Việt,Kinh doanh,"QA/Test, Điện Tử","Số 3 Trần Nhân Tôn, Phường 9, Quận 5, TPHCM","Tốt nghiệp chuyên ngành ô tô, điện tử… các ngà...","Mô tả sản phẩm: Các sản phẩm, thiết bị camera ...","10,000,000 - 15,000,000",1-3 năm,0.885638
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Nhân Viên Thủ Tục Pháp Lý Dự Án,Công Ty Cổ Phần Thép Hòa Phát Dung Quất,Sản xuất/Vận hành sản xuất,"Luật, Pháp Chế, Hành chính Văn phòng",Hà Nội,Nam tuổi từ 2535 tuổi;Trình độ chuyên môn: Kỹ ...,Thực hiện các thủ tục pháp lý dự án tại các cơ...,Thỏa thuận,Không yêu cầu kinh nghiệm,0.890110
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Giáo Viên Tiếng Nhật,Công Ty CP Phát Triển Nhân Lực Và Công Nghệ Th...,ĐÀO TẠO TIẾNG NHẬT,"Ngoại Ngữ, Giảng Viên","Trường CĐ LICKhu Phố Thượng, Đình Bảng, TP Từ ...",Tốt nghiệp đại học chính quy chuyên ngành tiê...,"1. Giảng dạy, quản lý học viên/sinh viênDạy ti...","10,000,000 - 15,000,000",Không yêu cầu kinh nghiệm,0.890247
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Giáo Viên Tiếng Anh Tại Biên Hòa,Công Ty TNHH Một Thành Viên Kiến Thức Việt,Giáo dục,"Ngoại Ngữ, Giảng Viên","161/1, Trương Định, P. Tân Mai","Tiếng anh lưu loát, phát âm chuẩn. Thái độ làm...",Giảng dạy theo chương trình của KTV English (K...,"5,000,000 - 10,000,000",Dưới 1 năm,0.890247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Huấn Luyện Viên Tiếng Anh Part - Time,Axcela Việt Nam,Giáo dục/Đào tạo,"Ngoại Ngữ, Trợ Giảng","123 Lý Chính Thắng, phường Võ Thị Sáu, Quận 3,...",Học chuyên ngành Tiếng Anh hoặc có các chứng c...,Thực hành giao tiếp tiếng anh 1 kèm 1 với học ...,"1,000,000 - 5,000,000",Không yêu cầu kinh nghiệm,0.891350
196,973260,Lê Thanh,Vận tải - Lái xe,24,Giáo Viên Tiếng Trung ( Online ),Trung Tâm Anh ngữ Wow English,Giáo dục/Đào tạo,"Ngoại Ngữ, Trợ Giảng","Cơ sở 2: Số 25 ngõ 183 Trần Đại Nghĩa, Hai Bà ...","Từ 18 tuổi trở lên.Kỹ năng giảng dạy, truyền đ...","Với vị trí này, bạn sẽ làm công việc gì?. Giản...","1,000,000 - 5,000,000",Không yêu cầu kinh nghiệm,0.891350
197,973260,Lê Thanh,Vận tải - Lái xe,24,Nhân Viên Thiết Kế Bim - Rebro,Công Ty Cổ Phần Jesco Asia,Thiết kế/Mỹ thuật,"Thiết Kế 3D, Thiết Kế 2D, Điện Tử","60 Trường Sơn, Phường 2, Tân Bình, Thành phố H...",Tốt nghiệp Cao Đẳng/ Đại Học các ngành có liên...,Công việc thực hiện bản vẽ Bim liên quan đến c...,"20,000,000 - 40,000,000",1-3 năm,0.892694
198,973260,Lê Thanh,Vận tải - Lái xe,24,"Nhân Viên Chăm Sóc Khách Hàng (Không Sale, Khô...",Bellsystem24 VietNam,Contact Center & BPO (Dịch Vụ Tổng Đài Cho Thuê ),"Phụ Kho, Tư Vấn Bán Hàng","Hội sở Techcombank, toà D, Vincom Trần Duy Hưn...","Nam/nữ từ đủ 18 tuổi 35 tuổi. Không ngọng, khô...","Tiếp nhận cuộc gọi, trả lời thắc mắc cho khách...","5,000,000 - 10,000,000",Không yêu cầu kinh nghiệm,0.893094


In [232]:
industry_recommendation_results = await Get_multi_recommendations(user_data, model, "phobert", "industry")
industry_recommendation_results

2024-12-16 01:10:13,894 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:10:13,902 INFO sqlalchemy.engine.Engine SELECT jobs_2.id, jobs_2.url, jobs_2.job_title, jobs_2.company_name, jobs_2.industry, jobs_2.company_address, jobs_2.job_description, jobs_2.job_requirements, jobs_2.benefits, jobs_2.salary, jobs_2.expire_date, jobs_2.employee_rank, jobs_2.job_skill, jobs_2.job_field, jobs_2.years_of_experience, jobs_2.job_address, jobs_2.job_title_vector, jobs_2.industry_vector, jobs_2.job_skill_vector, jobs_2.job_address_vector, jobs_2.industry_vector <=> $1 AS distance 
FROM jobs_2 
WHERE jobs_2.industry IS NOT NULL AND trim(regexp_replace(jobs_2.industry, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:10:13,904 INFO sqlalchemy.engine.Engine [generated in 0.00397s] ('[0.024326207116246223,0.07340653985738754,-0.003921078983694315,0.04226132482290268,0.08568193763494492,-0.04365711659193039,-0.029844509437680244,0. ...

,User ID,User Name,Industry,Age,Job Title Recommended,Company Name,Industry Recommended,Skills Recommended,Address Recommended,Job Requierements,Job Description,Job Salary,Job Years of Experience,Distance
0,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,QA Game Tester,"Nexon Networks Vina Co. Ltd,",Phần Mềm CNTT/Dịch vụ Phần mềm,"Game Testing, Quality Assurance, Manual Testin...","UOA TOWER, Tân Trào, Tân Phú, District 7, Thàn...",YÊU CẦU CÔNG VIỆC. Mới tốt nghiệp. Bằng cấp: C...,Chúng tôi tuyển dụng Nhân viên kiểm soát chất ...,Thương lượng,Không yêu cầu kinh nghiệm,0.893224
1,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,[ Hot Job ] Chuyên Gia Chính Sách và Phát Triể...,Công Ty Cổ Phần FPT,Phần Mềm CNTT/Dịch vụ Phần mềm,"Human Resources, Chính Sách Nhân Sự, C&B, Luật...","Hà Nội, Vietnam",Tốt nghiệp các chuyên ngành liên quan đến Kinh...,"Rà soát, xây dựng và cải tiến các chính sách n...",Thương lượng,5-10 năm,0.893224
2,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Senior Data Analyst,Capgemini Vietnam,Phần Mềm CNTT/Dịch vụ Phần mềm,"Python, SQL, Data Analysis, ETL, Computer Science","Centre Point, 106 Nguyễn Văn Trỗi, Phường 08, ...",Must have Technical Requirements / Qualificati...,"*** About the Role/position. In this role, you...",Thương lượng,1-3 năm,0.893224
3,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Java Backend Senior Developer,Capgemini Vietnam,Phần Mềm CNTT/Dịch vụ Phần mềm,"AWS, AWS, Backend, Backend, Java System, SQL","Centre Point, 106 Nguyễn Văn Trỗi, Phường 08, ...","Primary Skills:. ✓ Language: Java, SQL. ✓ Fram...","About the Role/position. In this role, you wil...",Thương lượng,3-5 năm,0.893224
4,964342,Nguyễn Hồng Duyên,Hành chính - Văn phòng,22,Data Engineer - Khối Dữ Liệu (Holt.02),Ngân Hàng Thương Mại Cổ Phần Quân Đội - MB,Phần Mềm CNTT/Dịch vụ Phần mềm,"Cloud Solution, Data Integration, Oracle","Quận Cầu Giấy, Hà Nội",Tốt nghiệp Đại học các chuyên ngành Công nghệ ...,". Thực hiện công việc tích hợp, phát triển dữ ...",Thương lượng,Không yêu cầu kinh nghiệm,0.893224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,973260,Lê Thanh,Vận tải - Lái xe,24,Global IT Sales Executive (Junior/Senior),MISA Jointstock Company,Phần Mềm CNTT/Dịch vụ Phần mềm,"Communication, Negotiation, Project Management...","Tòa nhà MISA Lô 5, Công viên Phần mềm Quang Tr...","Bachelor's degree in Business, Economics, Mana...","Job Description. Searching for agents, establi...","Trên 40,000,000",1-3 năm,0.895461
196,973260,Lê Thanh,Vận tải - Lái xe,24,Internship Tester (French),Capgemini Vietnam,Phần Mềm CNTT/Dịch vụ Phần mềm,"Java, Python, C#, Test, French","Centre Point, 106 Nguyễn Văn Trỗi, Phường 08, ...",Primary Skills:Programming Language: Any Objec...,"About the Role/position:. In this role, you wi...",Thương lượng,Không yêu cầu kinh nghiệm,0.895461
197,973260,Lê Thanh,Vận tải - Lái xe,24,Cobol Developer Intern,Capgemini Vietnam,Phần Mềm CNTT/Dịch vụ Phần mềm,"COBOL, English, French, Information Technology...","Centre Point, 106 Nguyễn Văn Trỗi, Phường 08, ...",Final year students or new graduates in Colleg...,***You ALL can apply to this internship with d...,Thương lượng,Không yêu cầu kinh nghiệm,0.895461
198,973260,Lê Thanh,Vận tải - Lái xe,24,Senior Azure/ Java Engineer,Capgemini Vietnam,Phần Mềm CNTT/Dịch vụ Phần mềm,"Azure DevOps, Azure Cloud, Maven, Java API, SQL","Centre Point, 106 Nguyễn Văn Trỗi, Phường 08, ...",Must have Technical Requirements / Qualificati...,"About the Role/position. In this role, you wil...",Thương lượng,Trên 10 năm,0.895461


In [ ]:
address_recommendation_results = await Get_multi_recommendations(user_data, model, "phobert", "location")
address_recommendation_results

2024-12-16 01:02:17,729 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:02:17,734 INFO sqlalchemy.engine.Engine SELECT jobs_3.id, jobs_3.url, jobs_3.job_title, jobs_3.company_name, jobs_3.industry, jobs_3.company_address, jobs_3.job_description, jobs_3.job_requirements, jobs_3.benefits, jobs_3.salary, jobs_3.expire_date, jobs_3.employee_rank, jobs_3.job_skill, jobs_3.job_field, jobs_3.years_of_experience, jobs_3.job_address, jobs_3.job_title_vector, jobs_3.industry_vector, jobs_3.job_skill_vector, jobs_3.job_address_vector, jobs_3.job_address_vector <=> $1 AS distance 
FROM jobs_3 
WHERE jobs_3.job_address IS NOT NULL AND trim(regexp_replace(jobs_3.job_address, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:02:17,736 INFO sqlalchemy.engine.Engine [cached since 336.7s ago] ('[-0.4315844178199768,0.06216150522232056,-0.555944561958313,-0.3563973903656006,-0.12172585725784302,0.03407549858093262,0.39096468687057495,-

""


In [ ]:
job_title_recommendation_results = await Get_multi_recommendations(user_data, model, "phobert", "job title")
job_title_recommendation_results

2024-12-16 01:02:19,071 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:02:19,074 INFO sqlalchemy.engine.Engine SELECT jobs_3.id, jobs_3.url, jobs_3.job_title, jobs_3.company_name, jobs_3.industry, jobs_3.company_address, jobs_3.job_description, jobs_3.job_requirements, jobs_3.benefits, jobs_3.salary, jobs_3.expire_date, jobs_3.employee_rank, jobs_3.job_skill, jobs_3.job_field, jobs_3.years_of_experience, jobs_3.job_address, jobs_3.job_title_vector, jobs_3.industry_vector, jobs_3.job_skill_vector, jobs_3.job_address_vector, jobs_3.job_title_vector <=> $1 AS distance 
FROM jobs_3 
WHERE jobs_3.job_title IS NOT NULL AND trim(regexp_replace(jobs_3.job_title, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:02:19,076 INFO sqlalchemy.engine.Engine [cached since 332.5s ago] ('[-0.10015001893043518,0.28205442428588867,-0.36834749579429626,-0.26571357250213623,-0.38623157143592834,0.29282432794570923,0.33689388632774353,-0.

""


In [ ]:
combined_recommendation_results = await Get_multi_recommendations(user_data, model, "phobert", "combined")
combined_recommendation_results

2024-12-16 01:02:20,936 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:02:20,943 INFO sqlalchemy.engine.Engine SELECT jobs_3.id, jobs_3.url, jobs_3.job_title, jobs_3.company_name, jobs_3.industry, jobs_3.company_address, jobs_3.job_description, jobs_3.job_requirements, jobs_3.benefits, jobs_3.salary, jobs_3.expire_date, jobs_3.employee_rank, jobs_3.job_skill, jobs_3.job_field, jobs_3.years_of_experience, jobs_3.job_address, jobs_3.job_title_vector, jobs_3.industry_vector, jobs_3.job_skill_vector, jobs_3.job_address_vector, jobs_3.job_title_vector <=> $1 AS job_title_distance, jobs_3.industry_vector <=> $2 AS industry_distance, jobs_3.job_skill_vector <=> $3 AS job_skill_distance, jobs_3.job_address_vector <=> $4 AS job_address_distance 
FROM jobs_3 
WHERE jobs_3.industry IS NOT NULL AND jobs_3.job_title IS NOT NULL AND jobs_3.job_skill IS NOT NULL AND jobs_3.job_address IS NOT NULL AND trim(regexp_replace(jobs_3.industry, $5::VARCHAR, $6::VARCHAR, $7::VARCHAR)) != $8::VAR

""


In [221]:
skill_recommendation_results.to_csv("results/phobert/skill_recommendation_results.csv", index=False)
industry_recommendation_results.to_csv("results/phobert/industry_recommendation_results.csv", index=False)
address_recommendation_results.to_csv("results/phobert/address_recommendation_results.csv", index=False)
job_title_recommendation_results.to_csv("results/phobert/job_title_recommendation_results.csv", index=False)
combined_recommendation_results.to_csv("results/phobert/combined_recommendation_results.csv", index=False)

## XLM RoBERTa

In [222]:
model = SentenceTransformer('xlm-roberta-base')

No sentence-transformers model found with name xlm-roberta-base. Creating a new one with mean pooling.


In [223]:
skill_recommendation_results = await Get_multi_recommendations(user_data, model, "xlm_roberta", "skill")
skill_recommendation_results

2024-12-16 01:02:31,574 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:02:31,579 INFO sqlalchemy.engine.Engine SELECT jobs_3.id, jobs_3.url, jobs_3.job_title, jobs_3.company_name, jobs_3.industry, jobs_3.company_address, jobs_3.job_description, jobs_3.job_requirements, jobs_3.benefits, jobs_3.salary, jobs_3.expire_date, jobs_3.employee_rank, jobs_3.job_skill, jobs_3.job_field, jobs_3.years_of_experience, jobs_3.job_address, jobs_3.job_title_vector, jobs_3.industry_vector, jobs_3.job_skill_vector, jobs_3.job_address_vector, jobs_3.job_skill_vector <=> $1 AS distance 
FROM jobs_3 
WHERE jobs_3.job_skill IS NOT NULL AND trim(regexp_replace(jobs_3.job_skill, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:02:31,581 INFO sqlalchemy.engine.Engine [cached since 368.8s ago] ('[-0.018804701045155525,0.015157408080995083,0.025009622797369957,0.02211725153028965,0.14088299870491028,0.023517217487096786,-0.019457638263702393,

""


In [224]:
address_recommendation_results = await Get_multi_recommendations(user_data, model, "xlm_roberta", "location")
address_recommendation_results

2024-12-16 01:02:33,786 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:02:33,790 INFO sqlalchemy.engine.Engine SELECT jobs_3.id, jobs_3.url, jobs_3.job_title, jobs_3.company_name, jobs_3.industry, jobs_3.company_address, jobs_3.job_description, jobs_3.job_requirements, jobs_3.benefits, jobs_3.salary, jobs_3.expire_date, jobs_3.employee_rank, jobs_3.job_skill, jobs_3.job_field, jobs_3.years_of_experience, jobs_3.job_address, jobs_3.job_title_vector, jobs_3.industry_vector, jobs_3.job_skill_vector, jobs_3.job_address_vector, jobs_3.job_address_vector <=> $1 AS distance 
FROM jobs_3 
WHERE jobs_3.job_address IS NOT NULL AND trim(regexp_replace(jobs_3.job_address, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:02:33,792 INFO sqlalchemy.engine.Engine [cached since 352.8s ago] ('[0.011592311784625053,0.09437571465969086,0.029185447841882706,-0.0574013814330101,0.09281326830387115,0.058053914457559586,-0.010301214642822

""


In [225]:
industry_recommendation_results = await Get_multi_recommendations(user_data, model, "xlm_roberta", "industry")
industry_recommendation_results

2024-12-16 01:02:35,115 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:02:35,120 INFO sqlalchemy.engine.Engine SELECT jobs_3.id, jobs_3.url, jobs_3.job_title, jobs_3.company_name, jobs_3.industry, jobs_3.company_address, jobs_3.job_description, jobs_3.job_requirements, jobs_3.benefits, jobs_3.salary, jobs_3.expire_date, jobs_3.employee_rank, jobs_3.job_skill, jobs_3.job_field, jobs_3.years_of_experience, jobs_3.job_address, jobs_3.job_title_vector, jobs_3.industry_vector, jobs_3.job_skill_vector, jobs_3.job_address_vector, jobs_3.industry_vector <=> $1 AS distance 
FROM jobs_3 
WHERE jobs_3.industry IS NOT NULL AND trim(regexp_replace(jobs_3.industry, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:02:35,122 INFO sqlalchemy.engine.Engine [cached since 365.7s ago] ('[0.024326207116246223,0.07340653985738754,-0.003921078983694315,0.04226132482290268,0.08568193763494492,-0.04365711659193039,-0.029844509437680244,0. .

""


In [226]:
job_title_recommendation_results = await Get_multi_recommendations(user_data, model, "xlm_roberta", "job title")
job_title_recommendation_results

2024-12-16 01:02:36,548 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:02:36,551 INFO sqlalchemy.engine.Engine SELECT jobs_3.id, jobs_3.url, jobs_3.job_title, jobs_3.company_name, jobs_3.industry, jobs_3.company_address, jobs_3.job_description, jobs_3.job_requirements, jobs_3.benefits, jobs_3.salary, jobs_3.expire_date, jobs_3.employee_rank, jobs_3.job_skill, jobs_3.job_field, jobs_3.years_of_experience, jobs_3.job_address, jobs_3.job_title_vector, jobs_3.industry_vector, jobs_3.job_skill_vector, jobs_3.job_address_vector, jobs_3.job_title_vector <=> $1 AS distance 
FROM jobs_3 
WHERE jobs_3.job_title IS NOT NULL AND trim(regexp_replace(jobs_3.job_title, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR)) != $5::VARCHAR ORDER BY distance 
 LIMIT $6::INTEGER
2024-12-16 01:02:36,552 INFO sqlalchemy.engine.Engine [cached since 349.9s ago] ('[-0.018961165100336075,0.05322413891553879,-0.006321473978459835,0.05661774054169655,0.057976894080638885,0.01587197370827198,-0.012791955843567848,

""


In [227]:
combined_recommendation_results = await Get_multi_recommendations(user_data, model, "xlm_roberta", "combined")
combined_recommendation_results

2024-12-16 01:02:38,554 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-16 01:02:38,563 INFO sqlalchemy.engine.Engine SELECT jobs_3.id, jobs_3.url, jobs_3.job_title, jobs_3.company_name, jobs_3.industry, jobs_3.company_address, jobs_3.job_description, jobs_3.job_requirements, jobs_3.benefits, jobs_3.salary, jobs_3.expire_date, jobs_3.employee_rank, jobs_3.job_skill, jobs_3.job_field, jobs_3.years_of_experience, jobs_3.job_address, jobs_3.job_title_vector, jobs_3.industry_vector, jobs_3.job_skill_vector, jobs_3.job_address_vector, jobs_3.job_title_vector <=> $1 AS job_title_distance, jobs_3.industry_vector <=> $2 AS industry_distance, jobs_3.job_skill_vector <=> $3 AS job_skill_distance, jobs_3.job_address_vector <=> $4 AS job_address_distance 
FROM jobs_3 
WHERE jobs_3.industry IS NOT NULL AND jobs_3.job_title IS NOT NULL AND jobs_3.job_skill IS NOT NULL AND jobs_3.job_address IS NOT NULL AND trim(regexp_replace(jobs_3.industry, $5::VARCHAR, $6::VARCHAR, $7::VARCHAR)) != $8::VAR

""


In [228]:
skill_recommendation_results.to_csv("results/xlm_roberta/skill_recommendation_results.csv", index=False)
industry_recommendation_results.to_csv("results/xlm_roberta/industry_recommendation_results.csv", index=False)
address_recommendation_results.to_csv("results/xlm_roberta/address_recommendation_results.csv", index=False)
job_title_recommendation_results.to_csv("results/xlm_roberta/job_title_recommendation_results.csv", index=False)
combined_recommendation_results.to_csv("results/xlm_roberta/combined_recommendation_results.csv", index=False)